In [1]:
from tool import *
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side

In [4]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

# Monitor 전업체
mnt_spdb = spdb.groupby('Division').get_group('MNT')
mnt_spdb = mnt_spdb[~mnt_spdb['From Site'].str.contains('PEGA')]
mnt_spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ]

,Planweek,Category,Created_at,SP
134,2024-02-12(W07),Latest26w,202402121709,918576.0
135,2024-02-12(W07),Latest26w,202402130921,920307.0
136,2024-02-12(W07),Latest26w,202402150843,924140.0
137,2024-02-19(W08),Final26w,202402201401,978177.0
138,2024-02-19(W08),Final52w,202402210937,1214772.0
139,2024-02-19(W08),Latest26w,202402191522,978881.0
140,2024-02-19(W08),Latest26w,202402200910,986457.0
141,2024-02-19(W08),Latest26w,202402221037,978177.0
142,2024-02-26(W09),Final26w,202402271452,1018070.0
143,2024-02-26(W09),Final52w,202402281001,1236694.0


# 모니터 주차별 컨테이너 개수 계산

In [12]:
i = 146
sp1 = mnt_spdb.merge(mnt_spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])

sp1.insert(6, 'Inch', sp1['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sp1 = pd.merge(sp1, site[['To Site', 'Subsidiary', 'Country']], how='left')

c1 = sp1['From Site'].str.contains('AUO')
c2 = sp1['From Site'].str.contains('TPV')
c3 = sp1['From Site'].str.contains('TCL')
c4 = sp1['From Site'].str.contains('BOEVT')
c5 = (sp1['Inch'] == '22')
c6 = (sp1['Inch'] == '24')
c7 = (sp1['Inch'] == '25')
c8 = (sp1['Inch'] == '27')
c9 = (sp1['Inch'] == '32')
c10 = sp1['Mapping Model.Suffix'].str.contains('27MC41D')

# Container 개수 계산(40FT 기준)
sp1.loc[c1 & c6 , 'Container Count'] = sp1.loc[c1 & c6, 'SP'] / 572 / 2 # AUO 24인치 Full Stuffing 기준 572대로 개수 산출
sp1.loc[c1 & c7, 'Container Count'] = sp1.loc[c1 & c7, 'SP'] / 416 / 2 # AUO 25인치 Full Stuffing 기준 416대로 개수 산출
sp1.loc[c1 & c8, 'Container Count'] = sp1.loc[c1 & c8, 'SP'] / 429 / 2 # AUO 27인치 Full Stuffing 기준 429대로 개수 산출
sp1.loc[c2 & c6, 'Container Count'] = sp1.loc[c2 & c6, 'SP'] / 800 / 2 # TPV 24인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c2 & c9, 'Container Count'] = sp1.loc[c2 & c9, 'SP'] / 224 / 2 # TPV 32인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c4 & c8, 'Container Count'] = sp1.loc[c4 & c8, 'SP'] / 480 / 2 # BOEVT 27인치 Full Stuffing 기준 429대로 개수 산출
sp1.loc[c4 & c9, 'Container Count'] = sp1.loc[c4 & c9, 'SP'] / 336 / 2 # BOEVT 32인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c5, 'Container Count'] = sp1.loc[c3 & c5, 'SP'] / 840 / 2 # TCL 22인치 Full Stuffing 기준 840대로 개수 산출
sp1.loc[c3 & c6, 'Container Count'] = sp1.loc[c3 & c6, 'SP'] / 800 / 2 # TCL 24인치 Full Stuffing 기준 800대로 개수 산출
sp1.loc[c3 & c8, 'Container Count'] = sp1.loc[c3 & c8, 'SP'] / 512 / 2 # TCL 27인치 Full Stuffing 기준 512대로 개수 산출
sp1.loc[c10, 'Container Count'] = sp1.loc[c10, 'SP'] / 256 / 2 # TCL 27인치 Costco 2 pack 모델 Full Stuffing 기준 256대로 개수 산출

c1 = sp1['Week Name'] >= '2024-03-04(W10)'
c2 = sp1['Week Name'] <= '2024-05-27(W22)'

price_term_map = {
    'EACM': 'CIF',
    'EAIL': 'CIP',
    'EAIN_DC': 'CIP',
    'EASL': 'CIP',
    'EASL_DO': 'EXW',
    'EATH': 'CIP',
    'EAVH_HCM': 'CIF',
    'EAVH_HN': 'CIF',
    'ECHK': 'CIP',
    'EEBN': 'CIP',
    'EEES': 'CIP',
    'EEDG': 'CIP',
    'EEFS': 'CIP',
    'EEHS': 'CIP',
    'EEIS': 'CIP',
    'EELA': 'CIP',
    'EEMK': 'CIP',
    'EEPL': 'CIP',
    'EEPL_CZ': 'CIP',
    'EEPT': 'CIP',
    'EERO': 'CIP',
    'EEUK': 'CIP',
    'EHAP_LGM': 'CIP',
    'EHAP_LGS': 'CIP',
    'EHAP_LGP': 'CIP',
    'EHAP_NZA': 'CIP',
    'EJJP': 'CIP',
    'ESPR': 'CFR',
    'EAML': 'CIP',
    'EATT': 'CIP',
    'EKHQ': 'FOB',
    'EMGF': 'CIP',
    'EMGF_ECHODUBAI_DO': 'CIP',
    'EMGF_MES_DO': 'CIP',
    'EMGF_NAAM_DO': 'CIP',
    'EMGF_PCI_DO': 'CIP',
    'EMSJ': 'CIP',
    'EMTK': 'CIF',
    'EMYK_HYE_DO': 'CIF',
    'ENCI': 'CIP',
    'ENMS': 'CIP',
    'ENUS_V': 'CIP',
    'ESCB': 'CFR',
    'ESCL_STG': 'CFR',
    'EMGF_ALTA_DO': 'CIF',
    'EESW':'CIP',
    'ECNT_P':'CIP',
    'ERRA_MINSK_DO':'CIP',
    'EMGF_TRIGON_DO':'CIP',
    'EMGF_MITSUMI_DO':'CIP',
}

sp1 = sp1[c1 & c2]
sp1['Price Term'] = sp1['To Site'].map(price_term_map)

sp1.isnull().sum() # 결측치가 없는지 확인

Planweek                0
Created_at              0
Division                0
From Site               0
Region                  0
To Site                 0
Inch                    0
Mapping Model.Suffix    0
Rep PMS                 0
Category                0
Frozen                  0
Week Name               0
SP                      0
Subsidiary              0
Country                 0
Container Count         0
Price Term              0
dtype: int64

In [13]:
result = sp1.fillna('TBD').pivot_table('Container Count', index=['From Site', 'To Site', 'Country', 'Region', 'Price Term'], columns='Week Name', aggfunc=sum).fillna(0).reset_index()
result.insert(1, 'Comp', 'BS')
result.insert(2, 'Div', 'IT(GMZ)')
result.insert(3, 'POL', 'CNYTN')
result.loc[result['From Site'].isin(['OS_AUO_CN_P', 'OS_TPV_CN_P']), 'POL'] = 'CNXMN'
result.loc[result['From Site'].isin(['OS_BOEVT_CN_P']), 'POL'] = 'CNSHA'
result.insert(8, 'DIO Y/N', 'N')
result.loc[result['To Site'].str.contains('_DO'), 'DIO Y/N'] = 'Y'
updated_time = datetime.datetime.now().strftime('%Y%m%d')
result.to_excel(f'D:/Downloads/{updated_time}_Monitor ODM container qty.xlsx', index=False)

In [14]:
print(f'''조연제 책임님,

수고 많으십니다.
첨부로 {sp1['Planweek'][0]} 기준 아웃소싱 모니터의 SP를 컨테이너 개수로 환산한 자료 공유드립니다.''')

조연제 책임님,

수고 많으십니다.
첨부로 2024-03-04(W10) 기준 아웃소싱 모니터의 SP를 컨테이너 개수로 환산한 자료 공유드립니다.
